In [1]:
from pathlib import Path
import pandas as pd
import src.data_manager as dm
import src.sense_similarity as sim

# Esercitazione 4
In questa esercitazione vedremo come utilizzare Nasari per un task di **sense similarity**. Nello specifico utilizzeremo la versione Nasari **embedded**, una versione in cui i concetti sono rappresentati in uno spazio embedded di 300 dimensioni.

Oltre al task principale ci occuperemo anche di annotare coppie di parole. L'esercitazione si divide in due task principali:
* Task 1: annotazione e valutazione dello score di similarità.
* Task 2: annotazione e valutazione dei sensi.

Come primo step analizziamo le 2 risorse lessicali principalmente utilizzate:

* Mapping lemma-to-synsets basato sul corpus *SemEval2017ITA*
* Nasari (versione embedded)

In [2]:
semeval = dm.SemEval(Path('data/SemEval17_IT_senses2synsets.txt'))

for lemma in ['agrume','bicicletta','prete']:
    senses_id = semeval.get_synsetsID('agrume')
    print(f"Il lemma: '{lemma}' ha i seguenti possibili babel synsets: {senses_id}")

Il lemma: 'agrume' ha i seguenti possibili babel synsets: {'bn:00019301n', 'bn:15303858n', 'bn:00019305n'}
Il lemma: 'bicicletta' ha i seguenti possibili babel synsets: {'bn:00019301n', 'bn:15303858n', 'bn:00019305n'}
Il lemma: 'prete' ha i seguenti possibili babel synsets: {'bn:00019301n', 'bn:15303858n', 'bn:00019305n'}


la classe `SemEval` rappresenta un **mapping** tra un lemma e alcuni dei synset disponibili in *BabelNet* associati ad esso.
La classe `Nasari` permette di accedere attraverso una semplice API alla risorsa lessicale omonima. 

Ad esempio dato un babel synset id possiamo recuperare la rappresentazione vettoriale associata:

In [3]:
nasari = dm.Nasari(Path('data/mini_NASARI.tsv'), mapper=semeval)
nasari.get_vector('bn:00019301n') # first sense of 'agrume' lemma

array([ 6.9381000e-04,  7.4659700e-03, -1.8382970e-02,  1.1926720e-01,
       -4.7373670e-02,  6.9858100e-03,  4.3458340e-02, -1.0989362e-01,
        2.8266000e-04,  1.5510333e-01,  9.4744760e-02, -1.5407147e-01,
       -8.8896630e-02,  6.6979250e-02, -2.0702154e-01,  1.3025508e-01,
       -1.7170539e-01,  2.1270724e-01, -3.2214940e-02,  2.0095530e-02,
       -8.8052400e-03,  4.4744130e-02,  5.5434610e-02,  3.3728700e-03,
       -3.2934440e-02, -7.8821210e-02, -1.1223518e-01,  4.3777560e-02,
       -2.4267820e-02,  6.6450460e-02, -4.6993650e-02, -2.2482930e-02,
        6.3626380e-02,  6.3381300e-03, -3.3713290e-02,  4.7530030e-02,
        2.6722000e-03, -1.8323416e-01,  1.0985690e-01,  9.4298880e-02,
        1.3884781e-01, -1.4735948e-01,  1.4661580e-02,  4.3326490e-02,
       -8.6036600e-02, -2.8626553e-01, -2.5982410e-02, -3.7649540e-02,
       -7.7904390e-02,  6.0011560e-02, -1.2072667e-01,  5.8185260e-02,
        7.1826710e-02, -5.4374200e-03, -2.5897080e-02,  7.5925530e-02,
      

Possiamo inoltre, sfruttando il mapper `semeval`, recuperare tutti i babel synsets e di conseguenza i vettori associati ad un lemma: 

In [4]:
lemma = 'agrume'
senses_id = nasari.get_lemma_senses(lemma) # just a wrapper to semeval mapper instance
vectors = nasari.get_lemma_vectors(lemma)
print(f"Il lemma: '{lemma}' ha i seguenti possibili babel synsets: {senses_id}\n")

print("Rappresentazione embedded:")
for sense_id, vector in zip(senses_id, vectors):
    print(f"{sense_id}:{vector}")

Il lemma: 'agrume' ha i seguenti possibili babel synsets: ['bn:00019301n', 'bn:15303858n', 'bn:00019305n']

Rappresentazione embedded:
bn:00019301n:[ 6.9381000e-04  7.4659700e-03 -1.8382970e-02  1.1926720e-01
 -4.7373670e-02  6.9858100e-03  4.3458340e-02 -1.0989362e-01
  2.8266000e-04  1.5510333e-01  9.4744760e-02 -1.5407147e-01
 -8.8896630e-02  6.6979250e-02 -2.0702154e-01  1.3025508e-01
 -1.7170539e-01  2.1270724e-01 -3.2214940e-02  2.0095530e-02
 -8.8052400e-03  4.4744130e-02  5.5434610e-02  3.3728700e-03
 -3.2934440e-02 -7.8821210e-02 -1.1223518e-01  4.3777560e-02
 -2.4267820e-02  6.6450460e-02 -4.6993650e-02 -2.2482930e-02
  6.3626380e-02  6.3381300e-03 -3.3713290e-02  4.7530030e-02
  2.6722000e-03 -1.8323416e-01  1.0985690e-01  9.4298880e-02
  1.3884781e-01 -1.4735948e-01  1.4661580e-02  4.3326490e-02
 -8.6036600e-02 -2.8626553e-01 -2.5982410e-02 -3.7649540e-02
 -7.7904390e-02  6.0011560e-02 -1.2072667e-01  5.8185260e-02
  7.1826710e-02 -5.4374200e-03 -2.5897080e-02  7.5925530e-0

Da notare l'ultimo synset: la versione nasari utilizzata è una versione ridotta, dunque è possibile avere dei sensi senza il corrispettivo vettore associato!

## Task 1: Semantic Word Similarity

Il task prevede nella prima parte l'annotazione manuale di coppie di parole (`file words_annotations.tsv`) con uno score di similarità compreso in $[0,4]$.
L'annotazione è stata effettuata seguendo gli stessi criteri utilizzati nel corpus *SemEval2017ITA*:

* 4: Very similar -- The two words are synonyms (e.g., midday-noon).
* 3: Similar -- The two words share many of the important ideas of their meaning but
include slightly different details. They refer to similar but not identical concepts (e.g., lionzebra).
* 2: Slightly similar -- The two words do not have a very similar meaning, but share a
common topic/domain/function and ideas or concepts that are related (e.g., house-window).
* 1: Dissimilar -- The two items describe clearly dissimilar concepts, but may share some
small details, a far relationship or a domain in common and might be likely to be found
together in a longer document on the same topic (e.g., software-keyboard).
* 0: Totally dissimilar and unrelated -- The two items do not mean the same thing and are
not on the same topic (e.g., pencil-frog


In [5]:
annotations = pd.read_csv(Path('data/words_annotations.tsv'), sep='\t')
annotations.head()

,lemma1,lemma2,score
0,recessione,PIL,2.5
1,Cesare,Giulio Cesare,4.0
2,paziente,sessione,3.0
3,comportamentismo,terapia,2.9
4,imperatore,costituzione,2.6


Una volta conclusa la fase di annotazione, gli score di similarità ottenuti rappresentano le annotazioni "gold standard" con cui possiamo confrontare i risultati ottenuti da un algoritmo. Nel caso specifico, per confrontare la similirità semantica utilizziamo la metrica *cosine similarity* su i vettori Nasari embedded.

Dato il fenomeno della polisemia, ad uno specifico lemma può essere associato più di un vettore, per questo motivo calcoliamo la massima similarità tra tutte le possibili coppie di sensi tra il primo e il secondo lemma della coppia, in formula:

$$ \operatorname{sim}(w_1, w_2) = \operatorname*{max}_{\substack{s_i \in \operatorname{Senses}(w_1)\\ s_j \in \operatorname{Senses}(w_2) }} \operatorname{sim}(s_i,s_j)$$

Nello nostro come funzione di simlarità viene utilizzata la similarità del coseno:

$$ \operatorname{sim}(s_i,s_j) = \frac{s_i\cdot s_j}{||s_i||\cdot||s_j||}$$

dove $s_i, s_j$ sono rispettivamente tutti i possibili sensi del lemma $w_1$ e $w_2$.



In [6]:
scores = annotations.copy()
scores['system_score'] = scores.apply(lambda x: sim.sense_similarity_score(x['lemma1'], x['lemma2'], sim.cosine_similarity, nasari), 
                                                axis=1)
scores.head()

,lemma1,lemma2,score,system_score
0,recessione,PIL,2.5,0.898526
1,Cesare,Giulio Cesare,4.0,1.000000
2,paziente,sessione,3.0,0.481492
3,comportamentismo,terapia,2.9,0.629044
4,imperatore,costituzione,2.6,0.635566


Una volta ottenuti gli score per ogni coppia di lemmi, possiamo verificare la relazione che intercorre tra lo score gold standard e quello ottenuto dal sistema, utilizzando gli indici di correlazione di Pearson e Spearman

In [7]:
scores.corr(method='pearson').loc['score','system_score']

0.6720570416841988

In [8]:
scores.corr(method='spearman').loc['score','system_score']

0.7003267480858936

## Task 2

Questo task consiste nell'annotare i termini con i rispettivi sensi e successivamente, seguendo la stessa metodologia del task 1,  valutare l'**accuratezza** del sistema rispetto alle annotazioni effettuate.


Partendo dalla stessa lista di coppie di lemmi estratti nel task 1, il processo prevede di annotare ciascun lemma con il corrispettivo Babel synset ID e una lista di termini che costituiscono un contesto di disambiguazione.

In [9]:
senses_annotations = pd.read_csv(Path('data/senses_annotations.tsv'), sep='\t')
senses_annotations.head()

,lemma1,senseID1,terms1,lemma2,senseID2,terms2
0,recessione,bn:00066516n,"{'recessione', 'Depressione_economica', 'Reces...",PIL,bn:00037570n,"{'prodotto_Interno_Lordo', 'Prodotto_interno_l..."
1,Cesare,bn:00014550n,"{'Gaius_Julius_Caesar', 'Gaio_Giulio_Cesare', ...",Giulio Cesare,bn:00014550n,"{'Gaius_Julius_Caesar', 'Gaio_Giulio_Cesare', ..."
2,paziente,bn:00001742n,"{'ruolo_del_paziente', 'paziente', 'ruolo_inte...",sessione,bn:00070690n,"{'tornata', 'sessione', 'seduta'}"
3,paziente,bn:00061017n,"{'malato', 'pazienti_ricoverati', 'Pazienti', ...",terapia,bn:00076842n,"{'terapeutica', 'terapia'}"
4,comportamentismo,bn:00009659n,"{'comportamentisti', 'analisi_del_comportament...",costituzione,bn:00022052n,"{'Costituzione_materiale', 'costituzione_codif..."


In [10]:
senses_scores = senses_annotations[['lemma1','lemma2',
                                    'senseID1','senseID2']].copy() # discard terms list columns

senses_scores['system_senseID1'] = senses_scores.apply(lambda x: sim.sense_similarity(x['lemma1'], x['lemma2'], sim.cosine_similarity, nasari)[1], 
                                                        axis=1) # [1] take the sense of the first lemma
senses_scores['system_senseID2'] = senses_scores.apply(lambda x: sim.sense_similarity(x['lemma1'], x['lemma2'], sim.cosine_similarity, nasari)[2], 
                                                        axis=1) # [2] take the sense of the second lemma

In [11]:
senses_scores.head()

,lemma1,lemma2,senseID1,senseID2,system_senseID1,system_senseID2
0,recessione,PIL,bn:00066516n,bn:00037570n,bn:00066516n,bn:00037570n
1,Cesare,Giulio Cesare,bn:00014550n,bn:00014550n,bn:00014550n,bn:00014550n
2,paziente,sessione,bn:00001742n,bn:00070690n,bn:00001742n,bn:03751534n
3,paziente,terapia,bn:00061017n,bn:00076842n,bn:00061017n,bn:00076842n
4,comportamentismo,costituzione,bn:00009659n,bn:00022052n,bn:00009659n,bn:00059480n


Come si può notare dall'output, in 4 casi su i 5 mostrati il sistema predice lo stesso senso per il lemma 1, per il lemma 2 solo in 3 cas su 5.

Analizziamo ora l'accuracy totale considerando i singoli lemmi e rispetto alla coppia:

In [12]:
def accuracy(true, predicted):
    correct_predictions = sum([true_sense.lower() == predicted_sense.lower() for true_sense, predicted_sense in zip(true, predicted)
    if true_sense and predicted_sense])
    return correct_predictions / len(true)

In [13]:
lemmas_gold_senses = pd.concat([senses_scores['senseID1'], senses_scores['senseID2']], 
                               axis=0, ignore_index=True) # concat lemmas vertically

lemmas_system_senses = pd.concat([senses_scores['system_senseID1'], senses_scores['system_senseID2']],
                                axis=0, ignore_index=True) # concat lemmas vertically
accuracy_score = accuracy(lemmas_gold_senses, lemmas_system_senses)

print(f"Accuratezza sui singoli lemmi: {accuracy_score}")

Accuratezza sui singoli lemmi: 0.45


Per valutare l'accuracy della coppia utilizziamo un semplice trick, concateniamo i due ysnset dei singoli lemmi e valutiamo l'accuracy.

In [14]:
pairs_gold_senses = senses_scores['senseID1']+senses_scores['senseID2']
pairs_system_senses = senses_scores['system_senseID1'].str.cat(senses_scores['system_senseID2'], na_rep='None')

accuracy_score = accuracy(pairs_gold_senses, pairs_system_senses)

print(f"Accuratezza sulla coppia di lemmi: {accuracy_score}")

Accuratezza sulla coppia di lemmi: 0.3


## Risultati

In entrambi i task abbiamo cercato di stabilire la similarità semantica tra coppie di termini. Sebbene i task utilizzano lo stesso metodo, il loro obbiettivo è differente. Nel primo caso bisgna solo quantificare la similarità a livello semantico di due termini; il secondo è nettamente più difficile in quanto bisogna individuare in maniera puntuale il senso di un termine.

La forte correlazione ottenuta nel primo task ($\approx 0.67$ Pearson, $\approx 0.7$ Spearman) non deve trarre in inganno perché non indica che il sistema abbia individuato il senso corretto, infatti a supporto di questa osservazione, possiamo considerare i bassi valori di accuracy ottenuta $0.46$.

Con poca sorpresa, l'accuracy ottenuta considerando i lemmi **congiuntamente** si abbassa ulteriormente a $0.3$.

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=afb22156-bb61-4d65-847d-18db79c0d4d2' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>